# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [11]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [12]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [13]:
def execute_create_or_drop_query(query):
    """Execute a query for creating or dropping a table and handle errors."""
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [14]:
def execute_select_query(query):
    """Execute a query to select contents of a table and display the rows."""
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        
        # Convert named tuple into a list so we only see the values
        print (list(row))

# song_plays_by_session
    Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Our table will need artist, song, length, because that is the information that will be requested. We also need sessionId, itemInSession because that is how the results will be filtered.

* Partition Key:
    Should be sessionId so that our data is distributed by session rather than distributed by position in session (itemInSession).
* Primary Key:
    sessionId by itself will not be unique, but if we add itemInSession as a clustering column, we can ensure uniqueness.

In [15]:
execute_create_or_drop_query("""
    CREATE TABLE IF NOT EXISTS song_plays_by_session (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession)
    )
""")              

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_plays_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
execute_select_query("""
    SELECT artist, song, length
    FROM song_plays_by_session
    WHERE sessionId=338 AND itemInSession=4
""")

['Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875]


# song_plays_by_user_session
    Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Our table will need artist, song, firstName, lastName, because that is the information that will be requested. We also need userId, sessionId, itemInSession because that is how the results will be filtered and ordered.

* Partition Key:
    Should be userId so that our data is distributed by user which is the first filter that is applied in the query.
* Primary Key:
    In this case we are filtering on two keys and then ordering on a third key, so our PK should be the partition key as well as two clustering columns, sessionId, itemInSession.

In [18]:
execute_create_or_drop_query("""
    CREATE TABLE IF NOT EXISTS song_plays_by_user_session (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )
""")
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_plays_by_user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

In [19]:
execute_select_query("""
    SELECT itemInSession, artist, song, firstName, lastName
    FROM song_plays_by_user_session 
    WHERE userId=10 AND sessionId=182
""")

[0, 'Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz']
[1, 'Three Drives', 'Greece 2000', 'Sylvie', 'Cruz']
[2, 'Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz']
[3, 'Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz']


# users_by_song_listens
    Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Our table will need firstName, lastName, because that is the information that will be requested. We also need song, userId for our Primary Key.
* Partition Key:
    Should be song so that our data is distributed by song which is the filter that is applied in the query.
* Primary Key:
    add userId to the PK as a clustering column for two reasons:
    * I want to ensure a unique PK and (song, name) didn't seem sufficient
    * As a clustering column, it will ensure my results are ordered by id which is nice
If i instead wanted alphabetical results, I could use PRIMARY KEY (song, firstName, userId), which would still ensure uniqueness, but would order by firstName instead.

In [28]:
execute_create_or_drop_query("""
    CREATE TABLE IF NOT EXISTS users_by_song_listens (
        song text,
        userId int,
        firstName text,
        lastName text,
        PRIMARY KEY (song, userId)
    )
""")

file = 'event_datafile_new.csv'

# Insert expected data into users_by_song_listens
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song_listens (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))                  

In [29]:
execute_select_query("""
    SELECT userId, firstName, lastName
    FROM users_by_song_listens
    WHERE song='All Hands Against His Own'
""")

[29, 'Jacqueline', 'Lynch']
[80, 'Tegan', 'Levine']
[95, 'Sara', 'Johnson']


### Drop the tables before closing out the sessions

In [31]:
## TO-DO: Drop the table before closing out the sessions

In [30]:
execute_create_or_drop_query("DROP TABLE song_plays_by_session")

execute_create_or_drop_query("DROP TABLE song_plays_by_user_session")

execute_create_or_drop_query("DROP TABLE users_by_song_listens")

### Close the session and cluster connection¶

In [32]:
session.shutdown()
cluster.shutdown()